In [ ]:
import pandas as pd
import numpy as np
from random import randint
from matplotlib import pyplot as plt

# Importing the dataset
dataset = pd.read_csv("cm_dataset.csv")

# The function takes 2 points as input and output the euclidean distance between them
def calculate_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2)**2))

In [ ]:
# Stores the x and y values and points in np arrays
x = np.array(dataset.iloc[:, 0].values)
y = np.array(dataset.iloc[:, 1].values)
points = np.array([[x[i], y[i]] for i in range(1000)])

# defining a very distant point
distant_point = np.array([1e9, 1e9])
#Number of data points
num_points = len(x)
# Number of iterations that are to be performed
iterations = 20
# Number of clusters
num_clusters = 2
# Number of different initialisations
num_init = 5

In [ ]:
plt.scatter(x, y)
plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
plt.xlabel("Given Data in xy-plane")
plt.show()

In [ ]:
# Initialisation Starts here
z = np.zeros((num_init, num_points), dtype='int') # The matrix that stores the information of which point belongs to which cluster
means = np.zeros((num_init, num_clusters, 2)) # Means of the clusters
errors = np.zeros((iterations+1, num_init)) # Errors in each clusters
diff_init_clusters = [] # Clusters after random initialisation
updated_means = np.zeros((iterations+1, num_init, num_clusters, 2)) # Means updated after each iteration
for i in range(num_init):
    
    clusters = []
    num_cluster_points = np.zeros(num_clusters)
    for j in range(num_clusters):
        clusters.append([[], []])

    for j, point in enumerate(points):
        z[i, j] = randint(0, num_clusters-1) # random initialisation of the clusters
        clusters[z[i, j]][0].append(x[j])
        clusters[z[i, j]][1].append(y[j])
        num_cluster_points[z[i, j]] += 1
        means[i, z[i, j]] += point
    diff_init_clusters.append(clusters)

    for j, mean in enumerate(means[i]):
        if num_cluster_points[j] == 0:
            mean = distant_point
        else:
            mean /= num_cluster_points[j]

for i in range(num_init):
    for j in range(num_points):
        errors[0, i] += calculate_distance(means[i, z[i, j], :], points[j])**2 # calculating errors
# Initialisation ends here


# Lloyd's Algorithm starts here
# This is the step of the algorithm that updates the means
def update_mean(means, z, points):
     for i in range(num_init):
          temp = np.zeros((num_clusters, 2))
          points_in_cluster = np.zeros(num_clusters)
          for j, point in enumerate(points):
               min_dist = calculate_distance(means[i, z[i, j], :], points[j])
               for k1 in range(num_clusters):
                    if(min_dist > calculate_distance(means[i, k1, :], points[j])):
                         z[i, j] = k1
                         points_in_cluster[k1] += 1
                         min_dist = calculate_distance(means[i, k1, :], points[j])
               temp[z[i, j]] += point
          temp /= num_points
          means[i] = temp

# Iterations start here
count = 0
while count < iterations:
    updated_means[count] = means
    update_mean(means, z, points)
    for i in range(num_init):
         for j in range(num_points):
              errors[count+1, i] += calculate_distance(means[i, z[i, j], :], points[j])**2
    count += 1

diff_final_clusters = [] # Clusters after all iterations
for i in range(num_init):
    
    clusters = []
    for j in range(num_clusters):
        clusters.append([[], []])

    for j, point in enumerate(points):
        clusters[z[i, j]][0].append(x[j])
        clusters[z[i, j]][1].append(y[j])
        means[i, z[i][j]] += point
    
    for j, mean in enumerate(means[i]):
        if num_cluster_points[j] == 0:
            mean = distant_point
        else:
            mean /= num_cluster_points[j]

    # means /= num_points
    diff_final_clusters.append(clusters)

# Plotting Intialisation and after clustering and the error function we can see a great dip at the first iteration
fig, axes = plt.subplots(num_init, 3, figsize=(18, 4*num_init))
for i in range(num_init):
    for j in range(num_clusters):
        for k in range(2):
            axes[i][k].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
        axes[i][0].set_xlabel("Data Before K-means")
        axes[i][1].set_xlabel("Data After K-means")
        axes[i][0].scatter(np.array(diff_init_clusters[i][j][0], dtype='float64'), np.array(diff_init_clusters[i][j][1], dtype='float64'))
        axes[i][1].scatter(np.array(diff_final_clusters[i][j][0], dtype='float64'), np.array(diff_final_clusters[i][j][1], dtype='float64'))
        axes[i][2].plot(np.linspace(1, iterations+1, iterations+1), errors[:, i])
        axes[i][2].set_xticks(np.linspace(0, 20, 5, dtype='int'))
        axes[i][2].set_xlabel("Number of Iterations")
        axes[i][2].set_ylabel("Value of the Error")
        axes[i][2].ticklabel_format(scilimits=(0, 4))
plt.show()

In [ ]:
# This shows the initial and the final positions of the means of the clusters
fig, axes = plt.subplots(2, num_init, figsize=(num_init*4, 8))
for i in range(num_init):
    axes[0][i].scatter(x, y)
    axes[1][i].scatter(x, y)
    for j in range(num_clusters):
        axes[0][i].plot(updated_means[0, i, j, 0], updated_means[0, i, j, 1], 'r*', markersize=8)
        axes[1][i].plot(updated_means[iterations - 1, i, j, 0], updated_means[iterations - 1, i, j, 1], 'r*', markersize=8)
        axes[0][i].tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
        axes[1][i].tick_params(bottom=False, left=False, labelleft=False, labelbottom=False)
        axes[0][i].set_xlabel("Mean of Clusters before K-means")
        axes[1][i].set_xlabel("Mean of Clusters after K-means")

plt.show()

In [ ]:
# This shows how the mean of differnt clusters changes after each iteration and the blue star indicates the final after given number of iterations
fig, axes = plt.subplots(1, num_init, figsize=(4*num_init, 4))
for i in range(num_init):
    for k in range(num_clusters):
        for j in range(iterations):
            axes[i].plot(updated_means[j, i, k, :][0], updated_means[j, i, k, :][1], 'r*')
        axes[i].plot(updated_means[iterations - 1, i, k, :][0], updated_means[iterations - 1, i, k, :][1], 'b*')
        axes[i].set_xlabel("Position of Means with Iteration")